In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch
from torch.utils.data import Subset

In [8]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    #transforms.RandomCrop(32, padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])

train_data = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform_train
)

test_data = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform_test
)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=4)

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

Files already downloaded and verified
Files already downloaded and verified


In [9]:
'''# !!!SUBSET!!!

p = 1

train_subset_size = int(p * len(train_data))
train_indexes = torch.randperm(len(train_data))[:train_subset_size]

train_subset = Subset(train_data, train_indexes)
train_loader = DataLoader(train_subset, batch_size=32)'''

'# !!!SUBSET!!!\n\np = 1\n\ntrain_subset_size = int(p * len(train_data))\ntrain_indexes = torch.randperm(len(train_data))[:train_subset_size]\n\ntrain_subset = Subset(train_data, train_indexes)\ntrain_loader = DataLoader(train_subset, batch_size=32)'

In [10]:
class VGG(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential( # -> 3x32x32
            nn.Conv2d(3, 64, 3, stride=1, padding=1), # -> 64x32x32
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2), # -> 64x16x16

            nn.Conv2d(64, 128, 3, stride=1, padding=1), # -> 128x16x16
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2), # -> 128x8x8

            nn.Conv2d(128, 256, 3, stride=1, padding=1), # -> 256x8x8
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2), # -> 256x4x4

            nn.Conv2d(256, 512, 3, stride=1, padding=1), # -> 512x4x4
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2), # -> 512x2x2

            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),     # 512x1x1
        )

        self.classifier = nn.Sequential(
            nn.Linear(512*1*1, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 10),
        )

    def forward(self, x):
        x = self.main(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    

vgg = VGG().to(device)

In [11]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.Adam(vgg.parameters(), lr=1e-3)

In [12]:
for epoch in range(5):
    running_loss = 0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()
        outputs = vgg(inputs)

        loss = crit(outputs, labels)
        loss.backward()
        optim.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Finished Training


In [13]:
correct = 0
total = 0

vgg.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = vgg(images)

        _, pred = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
        
print(f'Accuracy on test data: {100 * correct / total:.2f}%')

Accuracy on test data: 78.24%
